In [ ]:
import os
import pandas as pd
import configparser 
from datetime import datetime, timedelta
from yahoo_fin import stock_info as si
import yfinance as yf
import psycopg2

class PGDatabase:
    def __init__(self, host, database, user, password):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.connection = psycopg2.connect(
            host = host,
            user = user,
            password = password,
            database = database, 
        )

        self.cursor = self.connection.cursor()
        self.connection.autocommit = True

    def post(self, query, args=()):
        try:
            self.cursor.execute(query, args)
        except Exception as err:
            raise print(repr(err))
        
config = configparser.ConfigParser()
config.read('config.ini')

SALES_PATH = config['files']['SALES_PATH']
COMPANIES = eval(config['Companies']['COMPANIES'])
HOST = 'localhost'
DATABASE = 'finance'
USER = 'postgres'
PASSWORD = 'tw1xtw1x'

sales_df = pd.DataFrame()
if os.path.exists(SALES_PATH):
    sales_df = pd.read_csv(SALES_PATH)
    #os.remove(SALES_PATH)

historical_d = {}

for company in COMPANIES:
    try:
        stock = yf.Ticker(company)
        historical_d[company] = stock.history(period="1d").reset_index()  # Исторические данные за 1 день
        #print(historical_d[company])
    except Exception as e:
        print(f"Ошибка при получении данных для {company}: {e}")

database = PGDatabase(
    host=HOST,
    database=DATABASE,
    user=USER,
    password=PASSWORD
)

for i, row in sales_df.iterrows():
    query = f"insert into sales values ('{row['dt']}', '{row['company']}', '{row['transaction_type']}', {row['amount']})"
    print(query)
    database.post(query,  )

for copmany, data in historical_d.items():
    for i, row in data.iterrows():
        query = f"insert into stock values ('{row['Date']}', '{copmany}', {row['Open']}, {row['Close']})"
        database.post(query,  )

           dt company transaction_type  amount
0  26-03-2025    TSLA              buy     132
1  26-03-2025    NVDA              buy     171
2  26-03-2025    PLTR              buy     169
3  26-03-2025    SMCI              buy     997
4  26-03-2025    MLGO              buy     979
5  26-03-2025    TSLA             sell     503
6  26-03-2025    NVDA             sell     703
7  26-03-2025    PLTR             sell     900
8  26-03-2025    SMCI             sell     583
9  26-03-2025    MLGO             sell      72
insert into sales values ('26-03-2025', 'TSLA', 'buy', 132)
insert into sales values ('26-03-2025', 'NVDA', 'buy', 171)
insert into sales values ('26-03-2025', 'PLTR', 'buy', 169)
insert into sales values ('26-03-2025', 'SMCI', 'buy', 997)
insert into sales values ('26-03-2025', 'MLGO', 'buy', 979)
insert into sales values ('26-03-2025', 'TSLA', 'sell', 503)
insert into sales values ('26-03-2025', 'NVDA', 'sell', 703)
insert into sales values ('26-03-2025', 'PLTR', 'sell', 900)
